In [8]:
# imports
import re
import os
import sys
import io
import json
from dotenv import load_dotenv
from openai import OpenAI
from pathlib import Path
from datetime import datetime
import requests
import subprocess
from IPython.display import Markdown, display, update_display

In [9]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [10]:
system_message = """You are a helpful assistant whose main purpose is to generate datasets for a given business problem."""

In [11]:
def get_user_prompt_tabular(business_problem, dataset_format, file_format, num_samples):
    
    user_message = f"""
    The business problem is: {business_problem}. \n
    The dataset is expected to be in {dataset_format}. 
    For the dataset types such as tabular or time series implement python code for creating the dataset.
    If the generated dataset contains several entities, i.e. products, users, write the output for these entities into separate files. 
    The dependencies for python code should include only standard python libraries such as numpy, pandas and built-in libraries. 
    The output dataset is stored as a {file_format} file and contains {num_samples} samples. \n    
    """

    return user_message

def get_user_prompt_text(business_problem, dataset_format, file_format):
    
    user_message = f"""
    The business problem is: {business_problem}. \n
    The dataset is expected to be in {dataset_format}. 
    For the text type return the generated dataset and the python code to write the output to the files.
    If the generated dataset contains several entities, i.e. products, users, write the output for these entities into separate files. 
    The dependencies for python code should include only standard python libraries such as numpy, pandas and built-in libraries. 
    The output dataset is stored as a {file_format} file. \n    
    """

    return user_message

def select_user_prompt(business_problem, dataset_format, file_format, num_samples):
    user_prompt = ""
    if dataset_format == "Text":
        user_prompt = get_user_prompt_text(business_problem, dataset_format, file_format)
    elif dataset_format in ["Tabular", "Time-series"]:
        user_prompt = get_user_prompt_tabular(business_problem, dataset_format, file_format, num_samples)
    return user_prompt


In [12]:
def stream_gpt(business_problem, dataset_format, file_format, num_samples):

    user_prompt = select_user_prompt(
                    business_problem, dataset_format, file_format, num_samples
                )
    stream = openai.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": system_message},
            {
                "role": "user",
                "content": user_prompt,
            },
        ],
        stream=True,
    )

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

    return response


def stream_claude(business_problem, dataset_format, file_format, num_samples):
    user_prompt = select_user_prompt(
                    business_problem, dataset_format, file_format, num_samples
                )
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[
            {
                "role": "user",
                "content": user_prompt,
            }
        ],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply
            print(text, end="", flush=True)
    return reply


def generate_dataset(business_problem, dataset_format, file_format, num_samples, model):
    if model == "GPT":
        result = stream_gpt(business_problem, dataset_format, file_format, num_samples)
    elif model == "Claude":
        result = stream_claude(business_problem, dataset_format, file_format, num_samples)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far
    return result

In [15]:

def extract_code(text):
    # Regular expression to find text between ``python and ``
    match = re.search(r"```python(.*?)```", text, re.DOTALL)

    if match:
        code = match.group(0).strip()  # Extract and strip extra spaces
    else:
        code = ""
        print("No matching substring found.")

    return code.replace("```python\n", "").replace("```", "")


def execute_code_in_virtualenv(text, python_interpreter=sys.executable):
    """
    Execute the given Python code string within the specified virtual environment.
    
    Args:
    - code_str: str, the Python code to execute.
    - venv_dir: str, the directory path to the virtual environment created by pipenv.
    """
    # Construct the full path to the Python interpreter in the virtual environment
    # python_interpreter = f"{venv_dir}/bin/python"

    # Check if executing within the specified virtual environment interpreter
    if not python_interpreter:
        raise EnvironmentError("Python interpreter not found in the specified virtual environment.")

    # Prepare the command to execute the code
    code_str = extract_code(text)
    command = [python_interpreter, '-c', code_str]

    # Execute the command
    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        print("Output:", result.stdout)
        print("Errors:", result.stderr)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while executing the code: {e}")
    return result.stdout

# Example usage
code_string = """
print('Hello from Pipenv virtual environment!')
"""
venv_directory = sys.executable  # replace with your actual virtualenv path
(execute_code_in_virtualenv(code_string, venv_directory))

No matching substring found.
Output: 
Errors: 


''

In [7]:
# Example string
text = """
Some text here 
```python
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Parameters
num_records = 100
start_date = datetime(2023, 1, 1)
item_ids = [f'item_{i}' for i in range(1, num_records+1)]

# Generate dates
dates = [start_date + timedelta(days=i) for i in range(num_records)]

# Generate random views and clicks
np.random.seed(42)  # For reproducibility
views = np.random.poisson(lam=100, size=num_records)  # Average 100 views
clicks = np.random.binomial(n=views, p=0.1)  # 10% click-through rate

# Calculate rank based on clicks (lower is better)
# You can also modify this function as per your ranking criteria
ranks = [sorted(clicks, reverse=True).index(x) + 1 for x in clicks]  # Rank 1 is highest

# Assemble the DataFrame
data = {
    'date': dates,
    'item_id': item_ids,
    'views': views,
    'clicks': clicks,
    'rank': ranks
}

df = pd.DataFrame(data)

# Save to CSV
df.to_csv('fashion_classified_ranking_dataset.csv', index=False)
print("Dataset generated and saved as 'fashion_classified_ranking_dataset.csv'")
```
 and more text here.
"""


In [8]:
execute_code_in_virtualenv(text, venv_directory)

Output: Dataset generated and saved as 'fashion_classified_ranking_dataset.csv'

Errors: 


"Dataset generated and saved as 'fashion_classified_ranking_dataset.csv'\n"